Обучить любую модель классификации на датасете IRIS до применения PCA (d = 2) и после него. 
Сравнить качество классификации по отложенной выборке

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
Y = iris.target 
print(Y.shape, X.shape)

(150,) (150, 4)


In [3]:
# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [4]:
# Найдем собственные векторы и собственные значения (англ. Eigenvalues)
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения в порядке убывания:
(437.7746724797988, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.10457072021055, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.013531335697195, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.107225464292886, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


In [5]:
#Оценим долю дисперсии, которая описывается найденными компонентами
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329987, 22.850761786701778, 3.6689218892828697, 0.5178709107154814]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


Таким образом, первая главная компонента описывает почти 73% информации, а первые две в сумме - 95.8%. 
В то же время последняя компонента описывает всего 0.5% 
и может быть отброжена без страха значительных потерь в качестве нашего анализа. 
Мы отбросим последние две компоненты, оставив первые две.

In [6]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [7]:
# Сформируем новую матрицу "объекты-признаки"
# W как матрица перехода между 2мя базисами
Z = X_.dot(W)

In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
XPCAreduced = pca.fit_transform(X_)

In [9]:
XPCAreduced[:10]

array([[-2.26470281,  0.4800266 ],
       [-2.08096115, -0.67413356],
       [-2.36422905, -0.34190802],
       [-2.29938422, -0.59739451],
       [-2.38984217,  0.64683538],
       [-2.07563095,  1.48917752],
       [-2.44402884,  0.0476442 ],
       [-2.23284716,  0.22314807],
       [-2.33464048, -1.11532768],
       [-2.18432817, -0.46901356]])

In [10]:
X_train_before, X_test_before, y_train_before, y_test_before = train_test_split(X_, Y, test_size=0.3)
X_train_after, X_test_after, y_train_after, y_test_after = train_test_split(Z, Y, test_size=0.3)

In [11]:
parameters = {
    'n_estimators': [10, 20, 50, 100],
    'max_depth': np.arange(5, 20),
}

clf = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=parameters,
    scoring='accuracy',
    cv=5,
)
clf.fit(X_train_before, y_train_before)
clf.best_params_

{'max_depth': 12, 'n_estimators': 10}

In [12]:
parameters = {
    'n_estimators': [10, 20, 50, 100],
    'max_depth': np.arange(5, 20),
}

clf = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=parameters,
    scoring='accuracy',
    cv=5,
)
clf.fit(X_train_after, y_train_after)
clf.best_params_

{'max_depth': 15, 'n_estimators': 10}

In [13]:
rfc_before = RandomForestClassifier(max_depth=10, n_estimators=20)
rfc_before.fit(X_train_before, y_train_before)
y_pred_before = rfc_before.predict(X_test_before)
print(accuracy_score(y_test_before, y_pred_before))

0.9555555555555556


In [14]:
rfc_after = RandomForestClassifier(max_depth=17, n_estimators=10)
rfc_after.fit(X_train_after, y_train_after)
y_pred_after = rfc_after.predict(X_test_after)
print(accuracy_score(y_test_after, y_pred_after))

0.8666666666666667


Качество немного ухудшилось